In [ ]:
import io
from minio import Minio
from minio.error import S3Error

# Set up source and target MinIO servers

# Replace the IP, access_key and secret_key with the Local system minio credentials
source_minio = Minio(
    "127.0.0.1:9000",
    access_key="g9Rf0UgUoHU0bk0c",
    secret_key="bFo6e6Roth4T2l9KZTUGnymzp8aw1EAl",
    secure=False  # Change to True for HTTPS
)

# Replace the IP, access_key and secret_key with the Target system minio credentials
target_minio = Minio(
    "10.2.130.116:9000",
    access_key="mIxhs4LsycBVEAGj",
    secret_key="6gCOTjV3oTKIJUupW5OZYN4rMunOJDUS",
    secure=False # Change to True for HTTPS
)

# specify the source and target buckets
source_bucket = 'testraghava'
target_bucket = 'testraghava'


if target_minio.bucket_exists(target_bucket):
       print("{} bucket exists".format(target_bucket))
else:
       target_minio.make_bucket(target_bucket)
       print("{} bucket created".format(target_bucket))
       
# iterate over each object in the source bucket and copy it to the target bucket
for obj in source_minio.list_objects(source_bucket, recursive=True):
    try:
        # read the data from the source object into a BytesIO object
        source_object = source_minio.get_object(source_bucket, obj.object_name)
        data = io.BytesIO(source_object.read())
        # calculate the length of the data in bytes
        length = data.getbuffer().nbytes
        
        # copy the object to the target bucket
        target_minio.put_object(target_bucket, obj.object_name, data, length=length)
        print(f"Copied {obj.object_name} to {target_bucket}/{obj.object_name}")
        
    except S3Error as e:
        print(f"Error copying {obj.object_name}: {e}")